In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
from itertools import combinations
import pickle
import config.config as conf

In [3]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select a.c_date as qna_creation_date \
     , a.posttypeid as posttypeid \
     , count(*) cnt \
  from ( \
        select  to_char(creationdate,'YYYY-MM-DD') as c_date, posttypeid \
          from posts \
         where creationdate between' 2013-11-30' and '2023-11-30' \
          and  posttypeid in ('1', '2') \
           ) a \
group by a.c_date, a.posttypeid \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

OperationalError: connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


In [4]:
df = pd.DataFrame(rows, columns = [
  'qna_creation_date' 
, 'posttypeid' 
, 'cnt'
])


In [5]:
# 전체 데이터셋 
with open('../../data/ba_chatgpt_df.pkl', 'wb') as f:
    pickle.dump(df, f)


In [6]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select \
                  x.q_id \
                , x.q_posttypeid \
                , x.q_acceptedanswerid \
                , x.q_parentid \
                , x.q_creationdate \
                , x.q_score \
                , x.q_viewcount \
                , x.q_owneruserid \
                , x.q_title \
                , x.q_tags \
                , x.q_answercount \
                , x.q_commentcount \
                , x.q_reputation \
                , b.id               as a_id \
                , b.posttypeid       as a_posttypeid \
                , b.acceptedanswerid as a_acceptedanswerid \
                , b.parentid         as a_parentid \
                , b.creationdate     as a_creationdate \
                , b.score            as a_score \
                , b.viewcount        as a_viewcount \
                , b.owneruserid      as a_owneruserid \
                , b.title            as a_title \
                , b.tags             as a_tags \
                , b.answercount      as a_answercount \
                , b.commentcount     as a_commentcount \
                , d.reputation       as a_reputation \
  from ( \
           select a.id               as q_id \
                , a.posttypeid       as q_posttypeid \
                , a.acceptedanswerid as q_acceptedanswerid \
                , a.parentid         as q_parentid \
                , a.creationdate     as q_creationdate \
                , a.score            as q_score \
                , a.viewcount        as q_viewcount \
                , a.owneruserid      as q_owneruserid \
                , a.title            as q_title \
                , a.answercount      as q_answercount \
                , a.commentcount     as q_commentcount \
                , c.reputation       as q_reputation \
                , replace(replace(lower(a.tags), '<', ''), '>', ' ')as q_tags \
           from posts a \
              , users c \
           where a.creationdate between '2021-10-01' and '2021-11-30' \
             and a.posttypeid = '1' \
             and a.owneruserid is not null \
             and c.id = a.owneruserid \
             and (replace(replace(lower(a.tags), '<', ''), '>', ' ') like '%python %') \
       )   x \
        , posts b \
        , users d \
where b.parentid = x.q_id \
  and b.posttypeid = '2' \
  and b.owneruserid is not null \
  and d.id = b.owneruserid \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [7]:
df_bf_c = pd.DataFrame(rows, columns = [
  'q_id' 
, 'q_posttypeid' 
, 'q_acceptedanswerid'
, 'q_parentid' 
, 'q_creationdate' 
, 'q_score' 
, 'q_viewcount' 
, 'q_owneruserid' 
, 'q_title' 
, 'q_tags' 
, 'q_answercount' 
, 'q_commentcount' 
, 'q_reputation' 
, 'a_id' 
, 'a_posttypeid' 
, 'a_acceptedanswerid' 
, 'a_parentid' 
, 'a_creationdate' 
, 'a_score' 
, 'a_viewcount' 
, 'a_owneruserid' 
, 'a_title' 
, 'a_tags' 
, 'a_answercount'
, 'a_commentcount'
, 'a_reputation' 
])

In [8]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select \
                  x.q_id \
                , x.q_posttypeid \
                , x.q_acceptedanswerid \
                , x.q_parentid \
                , x.q_creationdate \
                , x.q_score \
                , x.q_viewcount \
                , x.q_owneruserid \
                , x.q_title \
                , x.q_tags \
                , x.q_answercount \
                , x.q_commentcount \
                , x.q_reputation \
                , b.id               as a_id \
                , b.posttypeid       as a_posttypeid \
                , b.acceptedanswerid as a_acceptedanswerid \
                , b.parentid         as a_parentid \
                , b.creationdate     as a_creationdate \
                , b.score            as a_score \
                , b.viewcount        as a_viewcount \
                , b.owneruserid      as a_owneruserid \
                , b.title            as a_title \
                , b.tags             as a_tags \
                , b.answercount      as a_answercount \
                , b.commentcount     as a_commentcount \
                , d.reputation       as a_reputation \
  from ( \
           select a.id               as q_id \
                , a.posttypeid       as q_posttypeid \
                , a.acceptedanswerid as q_acceptedanswerid \
                , a.parentid         as q_parentid \
                , a.creationdate     as q_creationdate \
                , a.score            as q_score \
                , a.viewcount        as q_viewcount \
                , a.owneruserid      as q_owneruserid \
                , a.title            as q_title \
                , a.answercount      as q_answercount \
                , a.commentcount     as q_commentcount \
                , c.reputation       as q_reputation \
                , replace(replace(lower(a.tags), '<', ''), '>', ' ')as q_tags \
           from posts a \
              , users c \
           where a.creationdate between '2022-11-01' and '2022-12-30' \
             and a.posttypeid = '1' \
             and a.owneruserid is not null \
             and c.id = a.owneruserid \
             and (replace(replace(lower(a.tags), '<', ''), '>', ' ') like '%python %') \
       )   x \
        , posts b \
        , users d \
where b.parentid = x.q_id \
  and b.posttypeid = '2' \
  and b.owneruserid is not null \
  and d.id = b.owneruserid \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [9]:
df_md_c = pd.DataFrame(rows, columns = [
  'q_id' 
, 'q_posttypeid' 
, 'q_acceptedanswerid'
, 'q_parentid' 
, 'q_creationdate' 
, 'q_score' 
, 'q_viewcount' 
, 'q_owneruserid' 
, 'q_title' 
, 'q_tags' 
, 'q_answercount' 
, 'q_commentcount' 
, 'q_reputation' 
, 'a_id' 
, 'a_posttypeid' 
, 'a_acceptedanswerid' 
, 'a_parentid' 
, 'a_creationdate' 
, 'a_score' 
, 'a_viewcount' 
, 'a_owneruserid' 
, 'a_title' 
, 'a_tags' 
, 'a_answercount'
, 'a_commentcount'
, 'a_reputation' 
])

In [10]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select \
                  x.q_id \
                , x.q_posttypeid \
                , x.q_acceptedanswerid \
                , x.q_parentid \
                , x.q_creationdate \
                , x.q_score \
                , x.q_viewcount \
                , x.q_owneruserid \
                , x.q_title \
                , x.q_tags \
                , x.q_answercount \
                , x.q_commentcount \
                , x.q_reputation \
                , b.id               as a_id \
                , b.posttypeid       as a_posttypeid \
                , b.acceptedanswerid as a_acceptedanswerid \
                , b.parentid         as a_parentid \
                , b.creationdate     as a_creationdate \
                , b.score            as a_score \
                , b.viewcount        as a_viewcount \
                , b.owneruserid      as a_owneruserid \
                , b.title            as a_title \
                , b.tags             as a_tags \
                , b.answercount      as a_answercount \
                , b.commentcount     as a_commentcount \
                , d.reputation       as a_reputation \
  from ( \
           select a.id               as q_id \
                , a.posttypeid       as q_posttypeid \
                , a.acceptedanswerid as q_acceptedanswerid \
                , a.parentid         as q_parentid \
                , a.creationdate     as q_creationdate \
                , a.score            as q_score \
                , a.viewcount        as q_viewcount \
                , a.owneruserid      as q_owneruserid \
                , a.title            as q_title \
                , a.answercount      as q_answercount \
                , a.commentcount     as q_commentcount \
                , c.reputation       as q_reputation \
                , replace(replace(lower(a.tags), '<', ''), '>', ' ')as q_tags \
           from posts a \
              , users c \
           where a.creationdate between '2023-10-01' and '2023-11-30' \
             and a.posttypeid = '1' \
             and a.owneruserid is not null \
             and c.id = a.owneruserid \
             and (replace(replace(lower(a.tags), '<', ''), '>', ' ') like '%python %') \
       )   x \
        , posts b \
        , users d \
where b.parentid = x.q_id \
  and b.posttypeid = '2' \
  and b.owneruserid is not null \
  and d.id = b.owneruserid \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [11]:
df_af_c = pd.DataFrame(rows, columns = [
  'q_id' 
, 'q_posttypeid' 
, 'q_acceptedanswerid'
, 'q_parentid' 
, 'q_creationdate' 
, 'q_score' 
, 'q_viewcount' 
, 'q_owneruserid' 
, 'q_title' 
, 'q_tags' 
, 'q_answercount' 
, 'q_commentcount' 
, 'q_reputation' 
, 'a_id' 
, 'a_posttypeid' 
, 'a_acceptedanswerid' 
, 'a_parentid' 
, 'a_creationdate' 
, 'a_score' 
, 'a_viewcount' 
, 'a_owneruserid' 
, 'a_title' 
, 'a_tags' 
, 'a_answercount'
, 'a_commentcount'
, 'a_reputation' 
])

In [12]:
# 전체 데이터셋 
with open('../../data/df_bf_c_df.pkl', 'wb') as f:
    pickle.dump(df_bf_c, f)

# 전체 데이터셋 
with open('../../data/df_md_c_df.pkl', 'wb') as f:
    pickle.dump(df_md_c, f)


# 전체 데이터셋 
with open('../../data/df_af_c_df.pkl', 'wb') as f:
    pickle.dump(df_af_c, f)


In [25]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select \
                  x.q_id \
                , x.q_creationdate \
                , b.id               as a_id \
  from ( \
           select a.id               as q_id \
                , a.creationdate     as q_creationdate \
           from posts a \
              , users c \
          where a.creationdate between '2020-11-01' and '2023-11-30' \
             and a.posttypeid = '1' \
             and a.owneruserid is not null \
             and c.id = a.owneruserid \
       )   x \
LEFT JOIN posts b ON \
	x.q_id = b.parentid \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [26]:
df_not_answered = pd.DataFrame(rows, columns = [
  'q_id' 
, 'q_creationdate'  
, 'a_id' 
])

In [27]:
# 전체 데이터셋 
with open('../../data/df_not_answered.pkl', 'wb') as f:
    pickle.dump(df_not_answered, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select b.tags, a.creationdate \
  from (select * \
  from posts \
where posttypeid = '2' \
  and creationdate between '2022-11-29' and '2022-12-02') a \
LEFT JOIN posts b ON \
	a.parentid = b.id \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()